In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

import tensorflow as tf
# from tensorflow.keras.layers import Dropout
# from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
# Select training data and MAE threshold
load_filename = '../Training_Data/Sample_Training_Data_PH2.csv'
mae_threshold = 0.5

In [ ]:
data = pd.read_csv(load_filename)

X = data.iloc[:, :4].values
y = data.iloc[:, 5].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [ ]:
# The Neural Network is trained repeatedly until the MAE is lower than the threshold. 
# The training process will result in different accuracies because the stochastic gradient descent 
# initialises weights randomly and tries to optimise from there.

def create_and_train_model(X_train, y_train, X_val, y_val):

    model = tf.keras.Sequential([
        tf.keras.layers.Dense(64, activation='tanh', input_shape=(4,)),
        tf.keras.layers.Dense(64, activation='tanh'),
        tf.keras.layers.Dense(1)
    ])

    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])
    early_stopping = EarlyStopping(monitor='val_mae', patience=2, verbose=0, restore_best_weights=True)

    history = model.fit(X_train, y_train, epochs=100, validation_data=(X_val, y_val), verbose=0, callbacks=[early_stopping])

    return model, history

def evaluate_model(model, X_test, y_test):
    test_loss, test_mae = model.evaluate(X_test, y_test, verbose=0)
    return test_loss, test_mae

best_mae = float('inf')
best_model = None
attempts = 0

while best_mae >= mae_threshold:
    attempts += 1
    print(f"Training attempt {attempts}")

    model, history = create_and_train_model(X_train, y_train, X_val, y_val)
    test_loss, test_mae = evaluate_model(model, X_test, y_test)
    
    print(f"Test MAE: {test_mae}")
    
    if test_mae < best_mae:
        best_mae = test_mae
        best_model = model

results = pd.DataFrame({
    'Metric': ['Loss', 'Mean Absolute Error (MAE)'],
    'Value': [test_loss, best_mae]
})

print("Best model found:")
display(results)

In [ ]:
model.save('../Models/Sample_Model_PH2.keras')

In [ ]:
# Evaluate the model on the test set
test_loss, test_mae = model.evaluate(X_test, y_test, verbose=0)

results = pd.DataFrame({
    'Metric': ['Loss', 'Mean Absolute Error (MAE)'],
    'Value': [test_loss, test_mae]
})

display(results)

In [ ]:
plt.figure(figsize=(10, 5))

plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Training MSE')
plt.plot(history.history['val_loss'], label='Validation MSE')
plt.xlabel('Epoch')
plt.ylabel('MSE')
plt.title('Training and Validation MSE')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['mae'], label='Training MAE')
plt.plot(history.history['val_mae'], label='Validation MAE')
plt.xlabel('Epoch')
plt.ylabel('MAE')
plt.title('Training and Validation MAE')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
# Make predictions on the test set just to visualise
predictions = model.predict(X_test)

results_df = pd.DataFrame({
    'i_L': X_test[:, 0], 
    'v_C': X_test[:, 1],
    'i_L old': X_test[:, 2],
    'v_C_old': X_test[:, 3],
    'Predicted': predictions.flatten()
})

display(results_df)